# ✅1) Importing Libraries:

In [1]:
import streamlit as st
import os
from tensorflow.keras.models import load_model, Sequential
from tensorflow.keras.layers import Dense
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import nltk
from datetime import datetime
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
import string
import joblib
# Download NLTK resources
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\منه\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\منه\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\منه\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\منه\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# sia = SentimentIntensityAnalyzer()

In [3]:
# def classify_sentiment(word):
#     score = sia.polarity_scores(word)['compound']
#     if score >= 0.05:
#         return "positive"
#     elif score <= -0.05:
#         return "negative"
#     else:
#         return "neutral"

In [4]:
Positive_sentiments = [
    'positive', 'happiness', 'joy', 'love', 'amusement', 'enjoyment', 'admiration', 'affection', 'awe',
    'acceptance', 'adoration', 'anticipation', 'calmness', 'excitement', 'kind', 'pride', 'elation',
    'euphoria', 'contentment', 'serenity', 'gratitude', 'hope', 'empowerment', 'compassion', 'tenderness',
    'arousal', 'enthusiasm', 'fulfillment', 'reverence', 'curiosity', 'determination', 'zest', 'hopeful',
    'proud', 'grateful', 'empathetic', 'compassionate', 'playful', 'free-spirited', 'inspired', 'confident',
    'thrill', 'overjoyed', 'inspiration', 'motivation', 'satisfaction', 'blessed', 'appreciation', 'confidence',
    'accomplishment', 'wonderment', 'optimism', 'enchantment', 'intrigue', 'playfuljoy', 'mindfulness', 'dreamchaser',
    'elegance', 'whimsy', 'harmony', 'creativity', 'radiance', 'wonder', 'rejuvenation', 'coziness', 'adventure',
    'melodic', 'festivejoy', 'innerjourney', 'freedom', 'dazzle', 'artisticburst', 'culinaryodyssey', 'resilience',
    'immersion', 'spark', 'marvel', 'positivity', 'kindness', 'friendship', 'success', 'exploration', 'amazement',
    'romance', 'captivation', 'tranquility', 'grandeur', 'emotion', 'energy', 'celebration', 'charm', 'ecstasy',
    'colorful', 'hypnotic', 'connection', 'iconic', 'journey', 'engagement', 'touched', 'triumph', 'heartwarming',
    'solace', 'breakthrough', 'joy in baking', 'envisioning history', 'imagination', 'vibrancy', 'mesmerizing',
    'culinary adventure', 'winter magic', 'thrilling journey', "nature's beauty", 'celestial wonder', 'creative inspiration',
    'runway creativity', "ocean's freedom", 'whispers of the past', 'relief','happy','joyfulreunion','adrenaline'
]

Negative_sentiments = [
    'negative', 'anger', 'fear', 'sadness', 'disgust', 'disappointed', 'bitter', 'confusion', 'shame',
    'despair', 'grief', 'loneliness', 'jealousy', 'resentment', 'frustration', 'boredom', 'anxiety', 'intimidation',
    'helplessness', 'envy', 'regret', 'numbness', 'melancholy', 'ambivalence', 'bitterness', 'yearning', 'fearful',
    'apprehensive', 'overwhelmed', 'jealous', 'devastated', 'frustrated', 'envious', 'dismissive', 'heartbreak',
    'betrayal', 'suffering', 'emotionalstorm', 'isolation', 'disappointment', 'lostlove', 'exhaustion', 'sorrow',
    'darkness', 'desperation', 'ruins', 'desolation', 'loss', 'heartache', 'solitude', 'obstacle', 'sympathy',
    'pressure', 'renewed effort', 'miscalculation', 'challenge', 'sad', 'hate', 'bad','bittersweet', 'embarrassed'
]

Neutral_sentiments = [
    'neutral', 'surprise', 'indifference', 'pensive', 'reflection', 'contemplation','mischievous','suspense','nostalgia'
]

# ✅ **2) Reading Dataset,Modifications:**

In [5]:
data = pd.read_csv("sentimentdataset.csv")

data.rename(columns={'Sentiment (Label)': 'Target'}, inplace=True)

#removing any spaces and making all of them lower case
data['Target'] = data['Target'].str.strip()
data['Target'] = data['Target'].str.lower()
data['Source'] = data['Source'].str.strip()
data['Source'] = data['Source'].str.lower()
data['Country'] = data['Country'].str.strip()
data['Country'] = data['Country'].str.lower()

In [6]:
data.head()

,ID,Text,Target,Timestamp,User,Source,Topic,Retweets,Likes,Country,Year,Month,Day,Hour
0,0,Enjoying a beautiful day at the park! ...,positive,1/15/2023 12:30,User123,twitter,#Nature #Park,15,30,usa,2023,1,15,12
1,1,Traffic was terrible this morning. ...,negative,1/15/2023 8:45,CommuterX,twitter,#Traffic #Morning,5,10,canada,2023,1,15,8
2,2,Just finished an amazing workout! 💪 ...,positive,1/15/2023 15:45,FitnessFan,instagram,#Fitness #Workout,20,40,usa,2023,1,15,15
3,3,Excited about the upcoming weekend getaway! ...,positive,1/15/2023 18:20,AdventureX,facebook,#Travel #Adventure,8,15,uk,2023,1,15,18
4,4,Trying out a new recipe for dinner tonight. ...,neutral,1/15/2023 19:55,ChefCook,instagram,#Cooking #Food,12,25,australia,2023,1,15,19


# ✅ **3) Seeing The Unique Values:**

In [7]:
data["Target"].unique()

array(['positive', 'negative', 'neutral', 'anger', 'fear', 'sadness',
       'disgust', 'happiness', 'joy', 'love', 'amusement', 'enjoyment',
       'admiration', 'affection', 'awe', 'disappointed', 'surprise',
       'acceptance', 'adoration', 'anticipation', 'bitter', 'calmness',
       'confusion', 'excitement', 'kind', 'pride', 'shame', 'elation',
       'euphoria', 'contentment', 'serenity', 'gratitude', 'hope',
       'empowerment', 'compassion', 'tenderness', 'arousal', 'enthusiasm',
       'fulfillment', 'reverence', 'despair', 'grief', 'loneliness',
       'jealousy', 'resentment', 'frustration', 'boredom', 'anxiety',
       'intimidation', 'helplessness', 'envy', 'regret', 'curiosity',
       'indifference', 'numbness', 'melancholy', 'nostalgia',
       'ambivalence', 'determination', 'zest', 'hopeful', 'proud',
       'grateful', 'empathetic', 'compassionate', 'playful',
       'free-spirited', 'inspired', 'confident', 'bitterness', 'yearning',
       'fearful', 'apprehensiv

In [8]:
#checking the amount of occarunces for each sentiment
print("Positive : ",data['Target'].value_counts()['positive'])
print("Negative : ",data['Target'].value_counts()['negative'])
print("Neutral : ",data['Target'].value_counts()['neutral'])

Positive :  45
Negative :  4
Neutral :  18


# ✅ **4) Checking for Null Values:**

In [9]:
print(data.isnull().sum())

ID           0
Text         0
Target       0
Timestamp    0
User         0
Source       0
Topic        0
Retweets     0
Likes        0
Country      0
Year         0
Month        0
Day          0
Hour         0
dtype: int64


In [10]:
#Checking for Duplications
data.duplicated().sum()

0

# ✅ **5) Changing Target column to only have positive,negative,*neutral*:**

In [11]:
#data['Target'] = data['Target'].apply(classify_sentiment)

In [12]:
for word in data['Target']:
  if word in Positive_sentiments:
    data['Target'].replace(word,'Positive',inplace=True)
  elif word in Negative_sentiments:
    data['Target'].replace(word,'Negative',inplace=True)
  elif word in Neutral_sentiments:
    data['Target'].replace(word,'Neutral',inplace=True)

C:\Users\منه\AppData\Local\Temp\ipykernel_4956\966214921.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Target'].replace(word,'Positive',inplace=True)
C:\Users\منه\AppData\Local\Temp\ipykernel_4956\966214921.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example

In [13]:
#checking the amount of occarunces for each sentiment
print("Positive : " ,data['Target'].value_counts()['Positive'])
print("Negative : " ,data['Target'].value_counts()['Negative'])
print("Neutral : " ,data['Target'].value_counts()['Neutral'])

Positive :  475
Negative :  206
Neutral :  51


In [14]:
data.head()

,ID,Text,Target,Timestamp,User,Source,Topic,Retweets,Likes,Country,Year,Month,Day,Hour
0,0,Enjoying a beautiful day at the park! ...,Positive,1/15/2023 12:30,User123,twitter,#Nature #Park,15,30,usa,2023,1,15,12
1,1,Traffic was terrible this morning. ...,Negative,1/15/2023 8:45,CommuterX,twitter,#Traffic #Morning,5,10,canada,2023,1,15,8
2,2,Just finished an amazing workout! 💪 ...,Positive,1/15/2023 15:45,FitnessFan,instagram,#Fitness #Workout,20,40,usa,2023,1,15,15
3,3,Excited about the upcoming weekend getaway! ...,Positive,1/15/2023 18:20,AdventureX,facebook,#Travel #Adventure,8,15,uk,2023,1,15,18
4,4,Trying out a new recipe for dinner tonight. ...,Neutral,1/15/2023 19:55,ChefCook,instagram,#Cooking #Food,12,25,australia,2023,1,15,19


In [15]:
data[['Topic 1', 'Topic 2']] = data['Topic'].str.split(expand=True)
data = data.drop('Topic', axis=1)

#making both columns lower_case
data['Topic 1'] = data['Topic 1'].str.lower()
data['Topic 2'] = data['Topic 2'].str.lower()

#Converting Year, Month and Day column to one Date column
data['date'] = pd.to_datetime(data[['Year', 'Month', 'Day']])

data.drop(['Year', 'Month', 'Day', 'Timestamp', 'Hour', 'ID', 'User' , 'Likes' , 'Retweets'], axis=1,inplace =True)

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 732 entries, 0 to 731
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Text     732 non-null    object        
 1   Target   732 non-null    object        
 2   Source   732 non-null    object        
 3   Country  732 non-null    object        
 4   Topic 1  732 non-null    object        
 5   Topic 2  732 non-null    object        
 6   date     732 non-null    datetime64[ns]
dtypes: datetime64[ns](1), object(6)
memory usage: 40.2+ KB


In [17]:
data.head()

,Text,Target,Source,Country,Topic 1,Topic 2,date
0,Enjoying a beautiful day at the park! ...,Positive,twitter,usa,#nature,#park,2023-01-15
1,Traffic was terrible this morning. ...,Negative,twitter,canada,#traffic,#morning,2023-01-15
2,Just finished an amazing workout! 💪 ...,Positive,instagram,usa,#fitness,#workout,2023-01-15
3,Excited about the upcoming weekend getaway! ...,Positive,facebook,uk,#travel,#adventure,2023-01-15
4,Trying out a new recipe for dinner tonight. ...,Neutral,instagram,australia,#cooking,#food,2023-01-15


In [18]:
data["Target"].unique()

array(['Positive', 'Negative', 'Neutral'], dtype=object)

# ✅ **6) Encoding The Target column to Unique Values:**


In [19]:
#detecting the unique values of the object type
for col in data.select_dtypes('object').columns:
    print(f'{col} -- {data[col].nunique()}')

Text -- 707
Target -- 3
Source -- 3
Country -- 33
Topic 1 -- 372
Topic 2 -- 636


In [20]:
encoder = LabelEncoder()
cols = ["Target","Topic 1","Topic 2","Country","Source"]
for i in cols:
  data[i] = encoder.fit_transform(data[i])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 732 entries, 0 to 731
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Text     732 non-null    object        
 1   Target   732 non-null    int32         
 2   Source   732 non-null    int32         
 3   Country  732 non-null    int32         
 4   Topic 1  732 non-null    int32         
 5   Topic 2  732 non-null    int32         
 6   date     732 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int32(5), object(1)
memory usage: 25.9+ KB


In [21]:
#!pip install category_encoders

In [22]:
# from sklearn.preprocessing import OneHotEncoder
# from category_encoders import BinaryEncoder

In [23]:
#le = LabelEncoder()
#data['Target'] = le.fit_transform(data['Target'])
#data['Target'].unique()

In [24]:
#To minimize the values in the dataset we used the binary encoding & one hot encoding
#one hot encoding for the columns that has low number of unique values
#ohe = OneHotEncoder(sparse=False , drop = 'first')
#ohe_df = pd.DataFrame(ohe.fit_transform(data[['Source']]) , columns=ohe.get_feature_names_out())
#ohe_df

In [25]:
#ohe = OneHotEncoder(sparse=False , drop = 'first')
#ohe_df = pd.DataFrame(ohe.fit_transform(data[['Source']]) , columns=ohe.get_feature_names_out())
#ohe_df

In [26]:
#Binary encoding for the columns that has alot of unique values
#be = BinaryEncoder()
#be_df = be.fit_transform(data[['Country']])
#be_df

In [27]:
#data = pd.concat([data , ohe_df , be_df] , axis = 1)

# ✅ **7) Function To Encode the Date Column to Unix Timestamp:**


In [28]:
def to_unix_timestamp(date_string):
    date_object = datetime.strptime(date_string, '%m/%d/%Y')
    return int(date_object.timestamp())

In [29]:
data['date'] = data['date'].apply(lambda x: pd.to_datetime(x).strftime('%m/%d/%Y'))

data['date'] = data['date'].apply(to_unix_timestamp)

In [30]:
data['date'].head()

0    1673733600
1    1673733600
2    1673733600
3    1673733600
4    1673733600
Name: date, dtype: int64

# ✅ **7) Creating Correlation Graph:**


In [31]:
temp=data.drop(['Text'],axis=1)
sns.heatmap(temp.corr(),annot=True)

<Axes: >

# ✅ **8) Preprocessing:**


1. Handling Text Data

In [32]:
# Tokenization, removing punctuation, lowercasing, removing stopwords, and stemming or lemmatization
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def preprocess_text(text):
    # Tokenization
    tokens = word_tokenize(text)

    # Removing punctuation and lowercasing
    tokens = [word.lower() for word in tokens if word.isalnum()]

    # Removing stopwords
    tokens = [word for word in tokens if word not in stop_words]

    return tokens

In [33]:
# Apply preprocessing function to the 'Text' column
data['Text'] = data['Text'].apply(preprocess_text)

In [34]:
lemmatizer = WordNetLemmatizer()
from nltk.corpus import wordnet

def get_wordnet_pos(tag):
    tag = tag[0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

def lemmatize_column(x):

    lemmatizer = WordNetLemmatizer()
    lemmatized_entry = [lemmatizer.lemmatize(token, pos=get_wordnet_pos(tag)) for token, tag in nltk.pos_tag(x)]
    return lemmatized_entry

data["Text"] = data["Text"].apply(lemmatize_column)

In [35]:
def join_tokens(tokenized_column):
    return tokenized_column.apply(lambda x: ' '.join(x))

data['Text'] = join_tokens(data['Text'])

In [36]:
# Print the DataFrame to check the result
data.head(15)

,Text,Target,Source,Country,Topic 1,Topic 2,date
0,enjoy beautiful day park,2,2,32,242,401,1673733600
1,traffic terrible morning,0,2,5,354,361,1673733600
2,finish amaze workout,2,1,32,151,628,1673733600
3,excited upcoming weekend getaway,2,0,31,356,7,1673733600
4,try new recipe dinner tonight,1,1,0,87,168,1673733600
5,feel grateful little thing life,2,2,13,172,417,1673820000
6,rainy day call cozy blanket hot cocoa,2,0,5,282,93,1673820000
7,new movie release,2,1,32,236,369,1673820000
8,political discussion heat timeline,0,2,32,271,112,1673906400
9,miss summer vibe beach day,1,0,0,330,30,1673906400


# ✅ **9) Create numerical features from the text data that the model can understand:**


In [37]:
# Initialize the TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# Fit the vectorizer to the text data and transform it into TF-IDF vectors
tfidf_vectors = vectorizer.fit_transform(data['Text'])

# Convert the TF-IDF vectors into a DataFrame
tfidf_vector = pd.DataFrame(tfidf_vectors.toarray(), columns=vectorizer.get_feature_names_out())

# Save the DataFrame with TF-IDF vectors to a CSV file
tfidf_vector.to_csv('tfidf_vectors.csv', index=False)

print("DataFrame with TF-IDF vectors saved to 'tfidf_vectors.csv'")

DataFrame with TF-IDF vectors saved to 'tfidf_vectors.csv'


In [38]:
Data_Final = pd.concat([data, tfidf_vector], axis=1)

In [39]:
Data_Final.head()

,Text,Target,Source,Country,Topic 1,Topic 2,date,ablaze,abstract,abyss,...,yearbook,yearn,yearning,yet,york,young,zen,zero,zest,zestful
0,enjoy beautiful day park,2,2,32,242,401,1673733600,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,traffic terrible morning,0,2,5,354,361,1673733600,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,finish amaze workout,2,1,32,151,628,1673733600,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,excited upcoming weekend getaway,2,0,31,356,7,1673733600,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,try new recipe dinner tonight,1,1,0,87,168,1673733600,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
Data_Final.to_csv("NEWDATAPRE.csv")

# ✅ **10) Multinomial Naive Bayes Model:**


In [41]:
df2 = pd.read_csv('NEWDATAPRE.csv')
df2.drop(['Text','date','Topic 1','Topic 2','Source','Country'] , axis = 1 , inplace = True)
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 732 entries, 0 to 731
Columns: 2095 entries, Unnamed: 0 to zestful
dtypes: float64(2093), int64(2)
memory usage: 11.7 MB


In [42]:
from sklearn.model_selection import train_test_split
X = df2.drop(columns=['Target'] , axis = 1)
y = df2['Target']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=8)

In [43]:
#NAIIVE BAY'S MODEL IMPLEMENTATION
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, ConfusionMatrixDisplay
from sklearn.model_selection import cross_val_score

# Initialize and train Multinomial Naive Bayes classifier
clf = MultinomialNB(alpha = 0.001 , force_alpha = True)
clf.fit(X_train, y_train)
y_train_prd = clf.predict(X_train)
y_test_prd = clf.predict(X_test)
# Evaluate performance
precision = precision_score(y_test, y_test_prd, average='weighted')
recall = recall_score(y_test, y_test_prd, average='weighted')
f1 = f1_score(y_test, y_test_prd, average='weighted')

#print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
scores = cross_val_score(clf, X, y, cv=5)

print("train acc" , accuracy_score(y_true=y_train , y_pred=y_train_prd))
print("test acc"  ,  accuracy_score(y_true=y_test , y_pred=y_test_prd))

# Print cross-validation scores
#print("Cross-Validation Scores:", scores)
print("Mean Accuracy:", scores.mean())


Precision: 0.928726035868893
Recall: 0.9251700680272109
F1-score: 0.9177350060903482
train acc 0.9982905982905983
test acc 0.9251700680272109
Mean Accuracy: 0.7703196347031963


In [44]:
joblib.dump(clf, 'naive_bayes_model.pkl')

['naive_bayes_model.pkl']

# ✅ **11) Logistic Regression Model:**


In [45]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

In [46]:
logistic_data = pd.read_csv('NEWDATAPRE.csv')
logistic_data.info()
logistic_data.drop(['Text','date','Topic 2','Source','Country'] , axis = 1 , inplace = True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 732 entries, 0 to 731
Columns: 2101 entries, Unnamed: 0 to zestful
dtypes: float64(2093), int64(7), object(1)
memory usage: 11.7+ MB


In [47]:
from sklearn.model_selection import train_test_split
X = logistic_data.drop(columns=['Target'] , axis = 1)
y = logistic_data['Target']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=8)

In [48]:
LR = LogisticRegression(penalty = 'l2' , C =6, tol = 1e-5)

In [49]:
LR.fit(X_train , y_train)

d:\Python\envs\test\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=6, tol=1e-05)

In [50]:
y_train_prd = LR.predict(X_train)
y_test_prd = LR.predict(X_test)

In [51]:
print(accuracy_score(y_true=y_train , y_pred=y_train_prd))
print(accuracy_score(y_true=y_test , y_pred=y_test_prd))

0.9777777777777777
0.8435374149659864


In [52]:
joblib.dump(LR, 'LR_model.pkl')

['LR_model.pkl']

# ✅ **12) Support Vector Machines (SVM):**


In [53]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Load the dataset
data_svm = pd.read_csv("NEWDATAPRE.csv")
# Assuming 'Text' column contains textual data and 'Target' column contains the sentiment labels
X = data_svm['Text']
y = data_svm['Target']

# Convert text data to numerical representation using TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=1000)  # You can adjust max_features as needed
X_tfidf = tfidf_vectorizer.fit_transform(X)

# Now proceed with train-test split and fitting the classifier
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=80)

# Initialize SVM classifier
clf = svm.SVC(kernel='linear')

# Train the classifier
clf.fit(X_train, y_train)

# Predict on the testing set
y_pred_test = clf.predict(X_test)
y_pred_train = clf.predict(X_train)

# Calculate accuracy
accuracy_train = accuracy_score(y_train, y_pred_train)
accuracy_test = accuracy_score(y_test, y_pred_test)
print("Accuracy Train:", accuracy_train)
print("Accuracy Test:", accuracy_test)

Accuracy Train: 0.9675213675213675
Accuracy Test: 0.9591836734693877


In [54]:
joblib.dump(clf, 'SVM_model.pkl')

['SVM_model.pkl']

# ✅ **13)Recurrent Neural Networks (RNNs):**


In [55]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.model_selection import train_test_split
# from imblearn.over_sampling import SMOTE

# # Split the dataset into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(data['Text'], data['Target'], test_size=0.25, random_state=42)

# # Initialize the TF-IDF vectorizer
# vectorizer = TfidfVectorizer()

# # Fit the vectorizer to the text data and transform it into TF-IDF vectors
# X_train_tfidf = vectorizer.fit_transform(X_train)
# X_test_tfidf = vectorizer.transform(X_test)

# # Convert TF-IDF vectors to arrays
# X_train_array = X_train_tfidf.toarray()
# X_test_array = X_test_tfidf.toarray()

# # Apply SMOTE for oversampling
# smote = SMOTE(random_state=42)
# X_train_resampled, y_train_resampled = smote.fit_resample(X_train_array, y_train)

# # Define the RNN model architecture
# model = Sequential()
# model.add(Dense(128, input_dim=X_train_resampled.shape[1], activation='relu'))
# model.add(Dense(num_classes, activation='softmax'))

# # Compile the model
# model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# # Train the model
# epochs = 10
# batch_size = 64
# history = model.fit(X_train_resampled, y_train_resampled, epochs=epochs, batch_size=batch_size, validation_data=(X_test_array, y_test), verbose=2)

# # Evaluate the model
# _, accuracy = model.evaluate(X_test_array, y_test)
# print('Accuracy:', accuracy)


In [56]:
# # Function to make predictions
# def predict_sentiment(input_text, vectorizer):
#     # Transform input text into TF-IDF vectors using the provided vectorizer
#     text_tfidf = vectorizer.transform([input_text])

#     # Convert TF-IDF vectors to arrays
#     text_array = text_tfidf.toarray()

#     # Make predictions
#     predictions = model.predict(text_array)

#     # Get the predicted class (assuming single-label classification)
#     predicted_class = np.argmax(predictions[0])

#     # Map the predicted class back to its original sentiment label
#     sentiment_labels = {0: 'Negative', 1: 'Neutral', 2: 'Positive'}
#     predicted_sentiment = sentiment_labels[predicted_class]

#     return predicted_sentiment

# # Example usage
# input_text = "I fail  "
# # Assuming you have access to the fitted vectorizer object (let's call it 'vectorizer')
# predicted_sentiment = predict_sentiment(input_text, vectorizer)
# print("Predicted sentiment:", predicted_sentiment)
